### 1A.

In [1]:
import pandas as pd
import openpyxl

def extract_song_data(file_path):
    
    #Load
    wb = openpyxl.load_workbook(file_path)

    #Sheet names for the Report Page N Results sheets
    results_sheet_names = [sheet_name for sheet_name in wb.sheetnames if 'Report Page' in sheet_name]

    rows_list = []
    
    #Loop through sheets
    for sheet_name in results_sheet_names:
        sheet = wb[sheet_name]
        
        #Create dictionary keys
        headers = ['Artist', 'Song Name']
        for row in sheet.iter_rows(min_row=12, min_col=1):
            for col, header in zip(row, sheet[11]):
                headers.append(header.value)
            break  
            
        #Find the artist and song name
        #A shortcut--it's not best practice to hardcode this. I would find this dynamically
        artist = sheet['C2'].value
        song_name = sheet['C3'].value
        
        #Loop through row for values
        for row in sheet.iter_rows(min_row=12, min_col=1):
            #Columns in each row
            cols = []
            for col in row:
                cols.append(col.value)
            
            #Add cols to beginning
            cols.insert(0, song_name)
            cols.insert(0, artist)

            rows_list.append(cols)
    
#Dataframe & cleanup: less efficient, but easier to adjust for analyst requirements
    song_data = pd.DataFrame(rows_list, columns=headers)

    #Relabel values in 'Activity' column
    song_data['Activity'] = song_data['Activity'].replace({'Songs w/SES On-Demand': 'Consumption', 'Digital Song Sales': 'Sales','Streaming On-Demand Audio': 'Audio','Streaming On-Demand Video': 'Video'})
    
    #Remove 'ISRC/Barcode' column
    song_data = song_data.drop('ISRC/Barcode', axis=1)
    
    #Select columns containing 'Week' & others needed for analysis below
    song_data = pd.concat([song_data.filter(like='Week'), song_data[['Song Name', 'Artist', 'Activity','% Chg','YTD (11/22/2018)','ATD (11/22/2018)']]], axis=1)

       
    return song_data


### 1B.

In [2]:
import pandas as pd
import openpyxl

def compare_song_data(file_path):
    
    #Load
    wb = openpyxl.load_workbook(file_path)

    #Sheet names for the Report Page N Results sheets
    results_sheet_names = [sheet_name for sheet_name in wb.sheetnames if 'Report Page' in sheet_name]

    rows_list = []
    
    #Loop through sheets
    for sheet_name in results_sheet_names:
        sheet = wb[sheet_name]
        
        #Create dictionary keys
        headers = ['Artist', 'Song Name']
        for row in sheet.iter_rows(min_row=12, min_col=1):
            for col, header in zip(row, sheet[11]):
                headers.append(header.value)
            break  
            
        #Find the artist and song name
        #A shortcut--it's not best practice to hardcode this. I would find this dynamically
        artist = sheet['C2'].value
        song_name = sheet['C3'].value
        
        #Loop through row for values
        for row in sheet.iter_rows(min_row=12, min_col=1):
            #Columns in each row
            cols = []
            for col in row:
                cols.append(col.value)
            
            #Add cols to beginning
            cols.insert(0, song_name)
            cols.insert(0, artist)

            rows_list.append(cols)
    
#Dataframe & cleanup: less efficient, but easier to adjust for analyst requirements
    song_data = pd.DataFrame(rows_list, columns=headers)

    #Relabel values in 'Activity' column
    song_data['Activity'] = song_data['Activity'].replace({'Songs w/SES On-Demand': 'Consumption', 'Digital Song Sales': 'Sales','Streaming On-Demand Audio': 'Audio','Streaming On-Demand Video': 'Video'})
    
    #Remove 'ISRC/Barcode' column
    song_data = song_data.drop('ISRC/Barcode', axis=1)
    
    #Select columns containing 'Week' & others needed for analysis below
    song_data = pd.concat([song_data.filter(like='Week'), song_data[['Song Name', 'Artist','Activity','% Chg','YTD (11/22/2018)','ATD (11/22/2018)']]], axis=1)

#Entity Selection

    song_names = song_data['Song Name'].unique().tolist()
    song_artists = song_data['Artist'].unique().tolist()  

    #Get the song list from the Report Summary
    report_song_list = []
    summary_sheet = wb['Report Summary']

    #Clean
    #A shortcut--it's not best practice to hardcode this. I would find this dynamically
    songs = summary_sheet['b4'].value
    for song_info in songs.split('\n'):
        song_info_parts = song_info.split("(Artist:")
        song_name = song_info_parts[0].replace("Song: ", "").strip()
        artist_name = song_info_parts[1].replace(")", "").strip()

        if song_name in song_names and artist_name not in song_artists:
            print("Warning:", song_name, "by", artist_name, "is NOT in the song_data.")  


In [3]:
compare_song_data(file_path='Trend Report Sample.xlsx')

### 1C. 

In [4]:
df = extract_song_data(file_path='Trend Report Sample.xlsx')

In [5]:
#What is the average number of audio streams for the current week in this set of songs?

#Activity='Audio'
audio_data= df[(df['Activity'] == 'Audio')]

#Avg for current week
avg_sales= audio_data['Week 47 2018 11/16/2018 - 11/22/2018'].mean()

print('The average number of audio streams for the current week is:', avg_sales)

The average number of audio streams for the current week is: 394677.9


In [6]:
#Which track has the 2nd highest consumption for the current week?

#Activity='Consumption'
consumption_data= df[(df['Activity'] == 'Consumption')]

#2nd highest value for current week
second_highest= consumption_data['Week 47 2018 11/16/2018 - 11/22/2018'].nlargest(2).iloc[-1]

#Song name
song_name= df.loc[df['Week 47 2018 11/16/2018 - 11/22/2018'] == second_highest, 'Song Name'].iloc[0]

print('The track has the 2nd highest consumption for the current week is:', song_name)

The track has the 2nd highest consumption for the current week is: Sit Next To Me


In [7]:
#A simplified consumption formula is Consumption = Sales + Audio/125 + Video/375.  
#For the song One Kiss’ activity-to-date data, Video accounts for what percentage of Consumption?

#Song='One Kiss'
one_kiss_data= df[(df['Song Name'] == 'One Kiss')]

#Activity='Video' & 'Consumption'
one_kiss_video_data= one_kiss_data.loc[(one_kiss_data['Activity'] == 'Video')]
one_kiss_con_data= one_kiss_data.loc[(one_kiss_data['Activity'] == 'Consumption')]

#Calc
consump = one_kiss_con_data['ATD (11/22/2018)'].sum()
vid_sum= one_kiss_video_data['ATD (11/22/2018)'].sum()
percent= round((vid_sum/375)/consump*100, 2)

print('For One Kiss, % Video/Consumption is:',percent)

For One Kiss, % Video/Consumption is: 17.88


In [8]:
#Which tracks increased their video streams by greater than 3% in the current week as compared to the previous week?

#Activity='Video'
video_data= df[(df['Activity'] == 'Video')]

#Sum video streams, group by current & last week
week_sum_by_song= video_data.groupby('Song Name')[['Week 46 2018 11/09/2018 - 11/15/2018', 'Week 47 2018 11/16/2018 - 11/22/2018']].sum()

#Filter to only include rows where current week> 3%
filtered_data= week_sum_by_song[(week_sum_by_song['Week 47 2018 11/16/2018 - 11/22/2018'] / week_sum_by_song['Week 46 2018 11/09/2018 - 11/15/2018'] > 0.03)]

#Song Names
print('Song names:', filtered_data.index.tolist())

Song names: ['Audio', 'Cherry Cola', 'One Kiss', 'Sit Next To Me', 'Strip', 'The Flute Song']


### 2.

    In the long term, if data providers are open to feedback about changing their song metadata, establishing a standardized naming convention for music data would be the ideal solution. However, for the short term, 
    there are a few ways to improve matches:

    *ISRC/Barcode: 
        The trend report sample had an ISRC/Barcode column. If this is a unique identifier, it can be used to 
        connect songs and artists without data cleaning and matching. After joining using these identifiers, it 
        would be easier to clean song/artist data knowing there is already a match. Concurrently, with this, I 
        would use a database.
    *SQL Database: 
        Retrieve the song/artist combinations from each site's API/other source of data, and store them in 
        separate lists or data structures. Create a table for each site that contains columns for the artist 
        name, song title, and any other relevant metadata. ISRC/Barcodes would be useful for this, but if that 
        is not available, develop a standardized naming convention that can be used and applied.	
    *Standardization: 
        Develop a set of rules or algorithms to match the artist/title combinations across different data sites.
        Ideally, data coming from Amazon, Apple, and Spotify has its own set of custom rules, however, Tiktok 
        and Youtube likely need a more complex set of rules due to live performances, leaked songs, 
        official/unofficial releases by individuals other than one corporation. Possible ways to standardize 
        would include: converting all characters to lowercase, remove special characters and spaces and 
        standardize them with an underscore/hyphen, trim unneeded words, standardize titles (Mrs, Mr) and
        different spellings (UK vs US).
    *Python libraries: 
        There are a couple Python libraries that can help with comparing titles–I’ve used fuzzywuzzy in the 
        past, which calculates a similarity score for two given strings.
    *Machine learning/similarity scores:
        Compare songs and artists based on their similarity scores.  I have extensive experience with this, as 
        I have developed and led projects that needed standardizing for job titles, company names, and 
        industries while working for Indeed.  I have built machine learning algorithms that are trained on large 
        datasets that assign a confidence score to each match based on their similarity score. A threshold is 
        used to filter out matches that fall below a certain level of confidence. Over time, an algorithm can 
        learn from past matches and continuously improve its matching capabilities over time. 
